# Clustering localities based on population sizes and locations

By Szymon Sacher, 2022

In this notebook I outline the procedure step-by-step. In a separate script I collect the 
steps into a function and run it for all countries.

In [1]:
import polars as pl
from polars import col, lit
import numpy as np
import pandas as pd

In [2]:
# Load the USA data

dt = pl.scan_csv("RevelioLabs_LocationSample_Nov22.csv"
        ).filter(
            col("country") == "united states"
        ).filter(
            col("population") != 0
        ).sort(
            col("longitude"), reverse= True
        ).with_row_count( # Primary Key
            "loc_id"
        ).with_column(
            pl.col("loc_id").cast(pl.Int32)
        ).collect()

# Assert unique and create a primary key
assert dt.unique().shape[0] == dt.shape[0]



In [3]:
'''
Notice that the mapping from City to MSA is noisy. For instance, Manhattan, NY is given "Manhattan KS MSA", an MSA in Kentucky.
What is more, some locations are repeated. For example, NYC is listed both as a whole and as individual boroughs.
I will treat it with care.
'''

dt.filter((col("state") == "new york") & ((col("city").is_in(["new york city", "manhattan", "bronx"])))).select(["city", "state", "msa"])

shape: (3, 3)
┌───────────────┬──────────┬─────────────────────────────────────┐
│ city          ┆ state    ┆ msa                                 │
│ ---           ┆ ---      ┆ ---                                 │
│ str           ┆ str      ┆ str                                 │
╞═══════════════╪══════════╪═════════════════════════════════════╡
│ bronx         ┆ new york ┆ New York-Northern New Jersey-Lon... │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ manhattan     ┆ new york ┆ Manhattan KS MSA                    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ new york city ┆ new york ┆ New York-Northern New Jersey-Lon... │
└───────────────┴──────────┴─────────────────────────────────────┘

## Compute distances

Below I compute the haversine distances between locations. Haversine distances account for curvature of the earth. A more satisfying approach would be to use actual travel time. For that I would have to use a service such as Google Maps API. This is unfortunately a paid service, so I would only do it if budget was provided.    

In [4]:
def distance(s_lat, s_lng, e_lat, e_lng):
    """
    Calculate the distance (KM) between two points on the earth (specified in decimal degrees)
    """

    # approximate radius of earth in km
    R = 6373.0

    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))

# Compute the haversine distance between each location
pairs = dt.select(["loc_id", "latitude", "longitude", "population"]).join(
        # Cross join with itself
        dt.select([
            col("loc_id").suffix("2"),
            col("latitude").suffix("2"),
            col("longitude").suffix("2"),
            col("population").suffix("2")
            ]),
        how="cross"
    ).filter( # Keep only pairs where loc_id < loc_id2  (to avoid duplicates) 
        col("loc_id") < col("loc_id2")
    ).with_column(  # Compute the distance
        distance(col("latitude"), col("longitude"), col("latitude2"), col("longitude2")).alias("distance")
    )

### Label which cities are in the same MSA

This is used later to tune parameters of optimization algorithm

In [5]:
# label which cities are in a MSA and which are in NONMETROPOLITAN

dt = dt.with_column(
        (col("msa").str.contains("MSA")).alias("is_MSA")
    )
msas = dt.filter(col("is_MSA") == True).select(["msa"]).unique().with_row_count("msa_id")
dt = dt.join(msas, on="msa", how="left")

pairs = pairs.join(
            dt.select(["loc_id", "msa_id"]), on="loc_id", how="left"
        ).join(
            dt.select(["loc_id", "msa_id"]), left_on="loc_id2", right_on = "loc_id", how="left", suffix= "2"
        ).with_column(
            ((col("msa_id") == col("msa_id2")) & col("msa_id").is_not_null()).alias("is_same_msa")
        ).sort(["loc_id", "loc_id2"])

# Defining Algorithm

The internal clustering algorithm simply amounts to finding connect components of an undirected graph. The nodes of this graph represent locations. The edges are formed whenever two conditions, governed by two free parameters are satisfied. The parameters are:

    `max_dist`: maximum distance between two locations that a link (edge) can be formed
    `min_pops`: minimum number of people in both locations 

Naturally, increasing both of the parameters will make the clustering sparser, meaning there will be more clusters (Quasi-MSAs), but each cluster will be smaller.

A researcher can have a preferred value for one or both of the parameters. However, we will optimize the parameters such that the clusters are `similar' to Census-defined MSAs. Specifically we will search for parameters such that a pair of cities are located in the same cluster if and only if the are in the same MSA. Since most pairs ARE NOT in the same MSAs, we will unweigh those that are.

We will then use these values for all cities internationally. The limitation, as indicated above, is that geographic haversine distance is a poor measure of connectivity as the travel time (much more important metric for defining MSAs) for a given distance varies a lot (both within USA and internationally), not to mention that it ignores features like bodies of water.

The clusters resulting from the algorithm are obtained in much data-intensive way than MSAs. They also use a easily explainable algorithm, rather than arbitrary judgment, which may be useful whenever expert judgment is not available.

In [6]:
import networkx as nx

def find_components(dist, max_dist, min_pops):
    """
    Find connected components in a distance matrix

    Parameters:
    dist (DataFrame): A DataFrame with columns "loc_id", "loc_id2", "distance", "population", "population2"
    max_dist (float): The maximum distance between two locations to be considered connected
    min_pops (int): The minimum population of a locations to be considered connected
    """

    # Filter the distance matrix
    dist = dist.filter(
        (col("distance") <= max_dist) & (col("population") >= min_pops) & (col("population2") >= min_pops)
    )

    # Create a graph
    g = nx.from_pandas_edgelist(dist, source="loc_id", target="loc_id2", edge_attr="distance")

    # Find connected components
    components = list(nx.connected_components(g))

    # Create a dataframe with the connected components
    components_df = pl.DataFrame(
        {
            "cc_id": [i for i, c in enumerate(components) for _ in range(len(c))],
            "loc_id": [l for c in components for l in c]
        }
    )

    # Merge the components with the original dataframe
    return components_df       

In [7]:
def loss(max_dist, min_pops, weight = 10):
    """
    Compute the loss function for a given max distance and min population
    as the fraction of pairs that are in the same US MSA and are also in the
    same connected component.

    Parameters:
    max_dist (float): The maximum distance between two locations to be considered connected
    min_pops (int): The minimum population of a locations to be considered connected
    weight (float): The relative weight to give in the loss function to the pairs that are in the same MSA
        (This is to encourage the connected components to be within the same MSA, given that the fraction of pairs
        that are in the same MSA is much smaller than the fraction of pairs that are not in the same MSA)
    
    """


    comps = find_components(pairs, max_dist, min_pops).with_column(
        col("loc_id").cast(pl.Int32)
    )

    dtt = dt.join(
                comps, on="loc_id", how="left"
            ).select(["loc_id", "cc_id"])

    dtt = dtt.join(
                dtt, how="cross", suffix = "2"
            ).filter(
                col("loc_id").cast(pl.Int32) < col("loc_id2").cast(pl.Int32)
            ).with_column(
                ((col("cc_id") == col("cc_id2")) & (col("cc_id").is_not_null())).alias("is_same_cc")
            ).sort(["loc_id", "loc_id2"])

    assert dtt.shape[0] == pairs.shape[0]

    same_msa = pairs["is_same_msa"].to_numpy()
    same_cc = dtt["is_same_cc"].to_numpy()

    loss = ((1 + weight * same_msa) * (same_msa != same_cc)).mean()

    return loss

In [8]:
'''
    We will minimize the loss using Nelder-Mead method.
    
    Unfortunately the loss function is not differentiable (and costly to evaluate).
    Nelder-Mead is a simple algorithm that does not require derivatives but it is
    slow. Hence we will terminate the optimization after (relatively few) iterations,
    in the interest of time.

'''

from scipy.optimize import minimize

def optimize_loss(max_dist_init, min_pops_init, maxiter = 200):
    """
    Optimize the loss function
    """
    res = minimize(
            fun = lambda x: loss(x[0], x[1]),
            x0 =  np.array([max_dist_init, min_pops_init]),
            method='Nelder-Mead',
            bounds = ((0, 30), (100, 500000)),
            options={'maxiter': maxiter, 'disp': True})
    
    return res

res = optimize_loss(5, 500, maxiter = 20)

c:\Users\szymo\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
#Print the results

print("Optimal max distance: ", res.x[0])
print("Optimal min population: ", res.x[1])
print("Loss: ", res.fun)

Optimal max distance:  14.060302734375
Optimal min population:  100.0
Loss:  0.05084438539548637


In [10]:
# Get final connected components based on the optimized parameters

comps = find_components(pairs, res.x[0], res.x[1]).with_column(
    col("loc_id").cast(pl.Int32)
)

# Merge the connected components with the original dataframe
dt = dt.join(
            comps, on="loc_id", how="left"
        )

# Save the results
dt.write_csv("connected_components_usa.csv")

# Save the optimized parameters

import json

with open("optimal_parameters.json", "w") as f:
    json.dump({"max_dist": res.x[0], "min_pops": res.x[1]}, f)

In [2]:
import polars as pl
dt = pl.read_csv("RevelioLabs_LocationSample_Nov22.csv")

In [7]:
# Save list of countries
countries = dt["country"].unique().to_list()

with open("countries.txt", "w") as f:
    f.write("\n".join(countries))
